In [1]:
import cv2
import dlib
import time
import numpy as np
from collections import Counter

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Indices for eye landmarks (Dlib 68-point model)
LEFT_EYE_INDICES = list(range(42, 48))
RIGHT_EYE_INDICES = list(range(36, 42))

def get_eye_aspect_ratio(eye_points):
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    return (A + B) / (2.0 * C)

def analyze_eye_tracking(frame, landmarks):
    left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in LEFT_EYE_INDICES])
    right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in RIGHT_EYE_INDICES])
    
    left_ear = get_eye_aspect_ratio(left_eye)
    right_ear = get_eye_aspect_ratio(right_eye)
    
    avg_ear = (left_ear + right_ear) / 2.0
    
    if avg_ear < 0.25:
        return "Drowsy or Low Engagement"
    elif avg_ear < 0.3:
        return "Neutral"
    else:
        return "Highly Engaged"

def get_most_frequent_engagement(engagement_list):
    if not engagement_list:
        return "No engagement data collected"
    return Counter(engagement_list).most_common(1)[0][0]

def main():
    cap = cv2.VideoCapture(0)
    print("Starting AI-driven eye-tracking system...")
    
    engagement_records = []
    start_time = time.time()
    duration = 15
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        
        for face in faces:
            landmarks = predictor(gray, face)
            engagement = analyze_eye_tracking(frame, landmarks)
            engagement_records.append(engagement)
        
        time.sleep(2)  # Simulate processing delay
        if time.time() - start_time >= duration:
            most_frequent_engagement= get_most_frequent_engagement(engagement_records)
            break  # Stop after 15 seconds
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    print(f"Most Frequent Engagement in the Last 15 seconds: {most_frequent_engagement}")
    cap.release()
    cv2.destroyAllWindows()
    import pandas as pd
    df=pd.read_csv("math_questions.csv")
    mapping={"Drowsy or Low Engagement":"Easy","Neutral":"Medium","Highly Engaged":"Hard"}
    def get_quiz(most_frequent_engagement,num_questions=10):
        difficulty=mapping.get(most_frequent_engagement,"Medium") 
        filt_df=df[df["Difficulty"]==difficulty] 
        selected_questions=filt_df.sample(n=min(num_questions,len(filt_df)),random_state=None).reset_index(drop=True)
        selected_questions.index+=1 
        return selected_questions
    quiz_ques = get_quiz(most_frequent_engagement)
    print("{} difficulty questions based on your mood".format(mapping[most_frequent_engagement]))
    print(quiz_ques[["Question"]])

if __name__ == "__main__":
    main()

2025-03-30 06:22:06.233 Python[41685:660831] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Starting AI-driven eye-tracking system...
Most Frequent Engagement in the Last 15 seconds: Neutral
Medium difficulty questions based on your mood
                          Question
1              What is 11 squared?
2   What is the square root of 60?
3              What is 84 squared?
4               What is 4 squared?
5                 What is 87 * 96?
6                 What is 24 * 35?
7                  What is 9 * 90?
8                 What is 74 * 27?
9                  What is 96 * 6?
10                What is 19 - 17?
